In [2]:
#Imports
import csv
import os
import glob
import json
import re
import string
import pandas as pd
import numpy as np
import scipy
from scipy import constants

#SciKit
from sklearn.datasets import load_iris
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.naive_bayes import GaussianNB

#NLTK
import nltk
from nltk.tokenize import TweetTokenizer

#Gensim
import gensim 
from gensim.models import Word2Vec, KeyedVectors


DATA PREPARATION

Before the content of the tweets can be analyzed, we need to acquire them and convert them into a form that can be fed into our feature engineering program. Thus, we will need to tokenize the Tweets. 

1. First, we will need to iterate through all of the .csv files of Tweets containing illicit material. 

2. The contents of these .csv files need to fed into the nltk TweetTokenizer() to be tokenized.

3. Tokenized Tweets will then appended to a new .json file. Assuming that Rudniy conducts his Tweet scraping in the same manner as the 'jailbait' Tweets, we will create a different json file for each "keyword".

4. In tandem, I need to create a parallel file of labels for all Tweets. I think it will be ok if this is simply a .json file. This is necessary in order to use sklearn's ML models. 

In [26]:
#This code gets the number of folders in a folder. This will be more important once we have more data. - FIX

path = "./Tweet_Datasets"

def count_folders(path):
  count = 0
  for folder in os.listdir(path):
    if os.path.isfile(os.path.join(path, folder)):
      count += 1
  return count

number_of_folders = count_folders(path)
print(number_of_folders)

0


In [10]:
#This code iterates through the #jailbait .csv files and appends all file names to a list 'file_names'

path = "./Tweet_Datasets/collections - #jailbait/*.csv"
file_names = []
for fname in glob.glob(path):
    file_names.append(fname)

In [16]:
#This code iterates through each file in 'file_names' and appends all Tweets to a single noise_amp list.

def returnFileNames(index):
    return file_names[index]

currentFile = 0
tweetText = []
tweetTime = []

while (currentFile < len(file_names)):
    file_path = returnFileNames(currentFile)

    if os.path.exists(file_path):
        with open(file_path, 'r') as rf:
            csv_reader = csv.reader(rf, delimiter=',')
            for row in csv_reader:
                if len(row) >= 2:
                    tweetText.append(row[1])
                    tweetTime.append(row[0])
    else:
        print(f"File not found: {file_path}")
    currentFile = currentFile + 1

print(tweetText)
print(tweetTime)


['text', "#morefun u can't be a snitch n hang out wit mark 24 hourz #JackBauer #jailbait", 'Nor any 23 y/o that would continue to see someone after finding out they were 16 #jailbait #PLL', 'cumsoline: It’s always funny to see people’s reactions when I tell them I’m 17. #jailbait http://t.co/4t4H1U4xgn', 'RT @GMayne94: @t_raww_7 jailbait..', "I ain't complainin' the BitchsBrew\nThere a lover who never knew shes Jailbait\nSteep the chase call the sweat\nYou steep the feet oh Jailbait", 'RT @Iain_MLaughlin: These girls should call themselves the Yewtree Dolls #jailbait', 'Naa jailbait', 'This episode of #XFactor is jailbait', '@bankofamerica charged with fraud by @SEC_News on RMBS. I wonder how the corporate charter will do in prison.\n#JailBait', 'hey jailbait calm down', '"America loves hot white jailbait ass" 😂😂😂 bruuuh', 'RT @OliviaViolet120: If you see a hot 16-year-old with a baby, does that mean she is a MILF or Jailbait?', 'RT @dirtyteens: #FriskyFriday #JailBait @Septawangsa @DR

In [17]:
#This code implements the nltk TweetTokenizer() and appends each tokenized Tweet to a list 'tokenized_tweets' 
#I still need to decide how to clean punctuation from Tweets without cleaning hashtags.

def cleanText(text):
    text_nonum = re.sub(r'\d+', '', text)
    text_nopunct = "".join([char.lower() for char in text_nonum if char not in string.punctuation]) 
    text_no_doublespace = re.sub('\s+', ' ', text_nopunct).strip()
    return text_no_doublespace

tokenized_tweets = []

for i in range(0, len(tweetText)):
    tweet_tokenizer = TweetTokenizer()
    tweet_tokens = tweet_tokenizer.tokenize(tweetText[i])
    tokenized_tweets.append(tweet_tokens)

In [18]:
print(tokenized_tweets)

[['text'], ['#morefun', 'u', "can't", 'be', 'a', 'snitch', 'n', 'hang', 'out', 'wit', 'mark', '24', 'hourz', '#JackBauer', '#jailbait'], ['Nor', 'any', '23', 'y', '/', 'o', 'that', 'would', 'continue', 'to', 'see', 'someone', 'after', 'finding', 'out', 'they', 'were', '16', '#jailbait', '#PLL'], ['cumsoline', ':', 'It', '’', 's', 'always', 'funny', 'to', 'see', 'people', '’', 's', 'reactions', 'when', 'I', 'tell', 'them', 'I', '’', 'm', '17', '.', '#jailbait', 'http://t.co/4t4H1U4xgn'], ['RT', '@GMayne94', ':', '@t_raww_7', 'jailbait', '..'], ['I', "ain't", 'complainin', "'", 'the', 'BitchsBrew', 'There', 'a', 'lover', 'who', 'never', 'knew', 'shes', 'Jailbait', 'Steep', 'the', 'chase', 'call', 'the', 'sweat', 'You', 'steep', 'the', 'feet', 'oh', 'Jailbait'], ['RT', '@Iain_MLaughlin', ':', 'These', 'girls', 'should', 'call', 'themselves', 'the', 'Yewtree', 'Dolls', '#jailbait'], ['Naa', 'jailbait'], ['This', 'episode', 'of', '#XFactor', 'is', 'jailbait'], ['@bankofamerica', 'charged', 

In [41]:
#This code writes all tokenized Tweets in the list 'tokenized_tweets' to a .csv file.

#with open('./Tweet_Datasets/Tokenized_Tweets/Tokenized_Tweets.csv', 'w', newline='') as file:
 #   writer = csv.writer(file)
 #   field = ["text"]
  #  writer.writerow(field)
   # for i in range(0, len(tokenized_tweets)):
    #    writer.writerow(tokenized_tweets[i])

In [20]:
#This code writes all tokenized Tweets in the list 'tokenized_tweets' to a .json file.

with open('./Tweet_Datasets/Tokenized_Tweets/Tokenized_Tweets.json', 'w') as f:
    tweet_data = []
    for i in range(0, len(tokenized_tweets)):
        tweet_entry = {
            "Tweet": tokenized_tweets[i],
            "Timestamp": tweetTime[i]
        }
        tweet_data.append(tweet_entry)

    json.dump(tweet_data, f, indent=6)


FEATURE ENGINEERING

Convert the Tweet text data into numerical features using appropriate text vectorization techniques, such as Bag-of-Words (BoW), Term Frequency-Inverse Document Frequency (TF-IDF), or word embeddings. Both SVM and Naive Bayes classifiers can work with these types of representations. 


Text Vectorization
Bag-of-Words (BoW)
a. Bag-of-Words is a simple text representation technique that converts text data into a matrix of word occurrences. Each row in the matrix represents a tweet, and each column represents a unique word from the entire corpus of tweets.

b. Use scikit-learn library
Term Frequency-Inverse Document Frequency (TF-IDF)
a. TF-IDF is another text representation technique that takes into account the importance of words based on their frequency in a particular tweet and across the entire corpus.

b. Use scikit-learn library
Word Embeddings
a. Word embeddings represent words as dense vector representations in a continuous vector space. We can use pre-trained word embeddings or train our own embeddings 

b. Libraries such as Word2Vec or FastText 



In [ ]:


X_train, X_test, y_train, y_test = train_test_split(tweets, labels, test_size=0.2, random_state=42)


In [ ]:
vectorizer = CountVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)